<a href="https://colab.research.google.com/github/tazar09/un_inputs/blob/main/18apr_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyPDF2
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=dc101be680fd62e2831b77a9b40075e4c76054e256eafeb25be9c80f858b927e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [4]:
import pandas as pd
pd.options.display.max_rows = 601

import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re

from PyPDF2 import PdfReader
from textblob import TextBlob

import time
from langdetect import detect

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [5]:
url = 'https://financing.desa.un.org/un-tax-convention/inputs'

In [6]:
def scrape_link(url):
    pdf_data = []
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')

      for link in soup.find_all('a', href=lambda href: href and href.endswith('.pdf')):
       text = link.text.strip()  # Extract link text (attribute)
       url = link.get('href')  # Extract PDF URL from 'href' attribute
       pdf_data.append({'text': text, 'url': url})

    # Create a Pandas DataFrame
      df = pd.DataFrame(pdf_data)
      return df

    else:
      print(f"Error: Website request failed with status code {response.status_code}")
      return none

In [7]:
def scrape_and_analyze_pdfs(url):
  # Data structure to store results
  data = []
  delay = 3

  # Scrape PDF links and attributes
  pdf_data = scrape_link(url = url)

  # Download and analyze each PDF
  if pdf_data is not None:
    for index, row in pdf_data.iterrows():
      pdf_url = row['url']
      text = row['text']

      # Download the PDF (replace with error handling)
      response = requests.get(pdf_url, stream=True)
      if response.status_code == 200:
        with open(f"pdf_{index+1}.pdf", 'wb') as f:
          for chunk in response.iter_content(1024):
            f.write(chunk)

        time.sleep(delay)
        # Extract text using PyPDF2 (replace with more advanced NLP techniques)
        reader = PdfReader(f"pdf_{index+1}.pdf")
        extracted_text = ""
        for page in reader.pages:
          extracted_text += page.extract_text()

        extracted_text = extracted_text.strip()
        extracted_text = re.sub(r'\s+', ' ', extracted_text)

        data.append({'text': text, 'url': pdf_url, 'extracted_text': extracted_text})

      # Remove downloaded PDF (optional)
      # os.remove(f"pdf_{index+1}.pdf")

  # Create a Pandas DataFrame
  df = pd.DataFrame(data)
  return df

In [8]:
# Call the function with your target URL (replace with your actual URL)
target_url = url
pdf_data = scrape_and_analyze_pdfs(target_url)

In [9]:
# pdf_data['extracted_text'] = pdf_data['extracted_text'].apply(lambda x: re.sub(r'\s+', ' ',x))
# pdf_data['extracted_text'] = pdf_data['extracted_text'].str.strip()

def extract_country(text):
  match = re.search(r"(?<=2024-03\/)([^_]+)(?=_Input)", text)
  return match.group(1) if match else None  # Handle cases where no match is found

pdf_data['country'] = pdf_data['url'].apply(extract_country)

pdf_data['country'] = pdf_data['country'].str.replace("%", ' ')
pdf_data['country'] = pdf_data['country'].str.replace("20", '')
pdf_data['country'] = pdf_data['country'].str.strip()


In [10]:
pdf_data = pdf_data[['text', 'country', 'url', 'extracted_text']]
pdf_data.head()

,text,country,url,extracted_text
0,African Group,African Group,https://financing.desa.un.org/sites/default/fi...,
1,Austria,Austria,https://financing.desa.un.org/sites/default/fi...,Invitation to provide inputs to the work of th...
2,Bahamas,Bahamas,https://financing.desa.un.org/sites/default/fi...,The Commonwealth of The Bahamas Input on UN Fr...
3,[ENG],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...
4,[RUS],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...


In [12]:
pdf.loc[pdf['text'] == '[ENG]', 'text'] = pdf['country'].where(pdf['text'] == '[ENG]')
pdf['sentiment'] = pdf['extracted_text'].apply(lambda x:TextBlob(x).sentiment.polarity)

In [13]:
def detect_language(text):
  try:
    return detect(text)
  except:
    return None  # Handle potential errors during language detection

pdf['language'] = pdf['extracted_text'].apply(detect_language)

In [14]:
pdf = pdf[['text', 'country', 'url', 'extracted_text', 'language', 'sentiment']]

In [15]:
pdf.head()

,text,country,url,extracted_text,language,sentiment
0,African Group,African Group,https://financing.desa.un.org/sites/default/fi...,,None,0.000000
1,Austria,Austria,https://financing.desa.un.org/sites/default/fi...,Invitation to provide inputs to the work of th...,en,0.127652
2,Bahamas,Bahamas,https://financing.desa.un.org/sites/default/fi...,The Commonwealth of The Bahamas Input on UN Fr...,en,0.116546
3,Belarus,Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,en,0.087142
4,[RUS],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,ru,0.077273


In [19]:
def analyze_sentiment(text):

  # Create VADER sentiment analyzer
  analyzer = SentimentIntensityAnalyzer()

  # Analyze sentiment
  sentiment = analyzer.polarity_scores(text)

  return sentiment

In [20]:
def sentiment_analysis_df(data):

  # Add new columns for sentiment scores
  data['sentiment'] = data['extracted_text'].apply(analyze_sentiment)
  data['compound'] = data['sentiment'].apply(lambda x: x['compound'])
  data['pos'] = data['sentiment'].apply(lambda x: x['pos'])
  data['neg'] = data['sentiment'].apply(lambda x: x['neg'])
  data['neu'] = data['sentiment'].apply(lambda x: x['neu'])  # Extract neutral score

  # Return the DataFrame with sentiment scores
  return data

In [26]:
pdf_data = pdf

In [27]:
pdf_sent = sentiment_analysis_df(pdf_data)  # Avoid modifying original DataFrame
pdf_sent.head()

,text,country,url,extracted_text,sentiment,language,compound,pos,neg,neu
0,African Group,African Group,https://financing.desa.un.org/sites/default/fi...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",None,0.0000,0.000,0.000,0.000
1,Austria,Austria,https://financing.desa.un.org/sites/default/fi...,Invitation to provide inputs to the work of th...,"{'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'comp...",en,0.9957,0.125,0.000,0.875
2,Bahamas,Bahamas,https://financing.desa.un.org/sites/default/fi...,The Commonwealth of The Bahamas Input on UN Fr...,"{'neg': 0.037, 'neu': 0.819, 'pos': 0.144, 'co...",en,0.9996,0.144,0.037,0.819
3,Belarus,Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,"{'neg': 0.012, 'neu': 0.903, 'pos': 0.085, 'co...",en,0.9969,0.085,0.012,0.903
4,[RUS],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,"{'neg': 0.0, 'neu': 0.967, 'pos': 0.033, 'comp...",ru,0.9810,0.033,0.000,0.967


In [28]:
# pdf_sent.to_excel('inputs_sent.xlsx')

In [29]:
df = pd.read_excel('/content/inputs_sent.xlsx')

In [46]:
def count_words(text):
  if isinstance(text, str):
    return len(text.split())
  else:
    return 0  # Or any other value you want for non-strings (e.g., np.NAN)

In [47]:
df['total_words'] = df['extracted_text'].apply(lambda x: count_words(x))

In [48]:
df.head()

,Unnamed: 0,text,country,url,extracted_text,sentiment,language,compound,pos,neg,neu,total_words
0,0,African Group,African Group,https://financing.desa.un.org/sites/default/fi...,NaN,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",NaN,0.0000,0.000,0.000,0.000,0
1,1,Austria,Austria,https://financing.desa.un.org/sites/default/fi...,Invitation to provide inputs to the work of th...,"{'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'comp...",en,0.9957,0.125,0.000,0.875,544
2,2,Bahamas,Bahamas,https://financing.desa.un.org/sites/default/fi...,The Commonwealth of The Bahamas Input on UN Fr...,"{'neg': 0.037, 'neu': 0.819, 'pos': 0.144, 'co...",en,0.9996,0.144,0.037,0.819,2075
3,3,Belarus,Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,"{'neg': 0.012, 'neu': 0.903, 'pos': 0.085, 'co...",en,0.9969,0.085,0.012,0.903,1305
4,4,[RUS],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,"{'neg': 0.0, 'neu': 0.967, 'pos': 0.033, 'comp...",ru,0.9810,0.033,0.000,0.967,1038


In [50]:
df.sort_values(by = 'total_words', ascending = False)

,Unnamed: 0,text,country,url,extracted_text,sentiment,language,compound,pos,neg,neu,total_words
55,55,UN Regional Commissions,UN Regional Commissions,https://financing.desa.un.org/sites/default/fi...,1 Regional Commissions’ inputs to the Ad Hoc C...,"{'neg': 0.039, 'neu': 0.835, 'pos': 0.126, 'co...",en,0.9999,0.126,0.039,0.835,4765
71,71,Civil Society Financing for Development Mechanism,Civil Society Financing for Development Mechanism,https://financing.desa.un.org/sites/default/fi...,"For the kind attention of: Mr. Ramy Youseff, C...","{'neg': 0.02, 'neu': 0.855, 'pos': 0.126, 'com...",en,0.9999,0.126,0.020,0.855,3539
54,54,UN Office of the High Commissioner for Human R...,UN Office of the High Commissioner for Human R...,https://financing.desa.un.org/sites/default/fi...,1 Submission from the UN Office of the High Co...,"{'neg': 0.044, 'neu': 0.825, 'pos': 0.131, 'co...",en,0.9997,0.131,0.044,0.825,2558
80,80,International Centre for Tax and Development (...,International Centre for Tax and Development ...,https://financing.desa.un.org/sites/default/fi...,Response to call for input on: “What are some ...,"{'neg': 0.033, 'neu': 0.895, 'pos': 0.072, 'co...",en,0.9972,0.072,0.033,0.895,2453
75,75,Gender and Tax Advocates,Submission on behalf of Gender and tax abvocates,https://financing.desa.un.org/sites/default/fi...,"1From: Sent: Friday, 15 March, 2024 11:33 AM T...","{'neg': 0.043, 'neu': 0.854, 'pos': 0.103, 'co...",en,0.9989,0.103,0.043,0.854,2338
96,96,"Financial Transparency Coalition, Center for E...",Financial Transparency Coalition 2C Center for...,https://financing.desa.un.org/sites/default/fi...,The need for climate mainstreaming in Internat...,"{'neg': 0.044, 'neu': 0.855, 'pos': 0.101, 'co...",en,0.9990,0.101,0.044,0.855,2268
35,35,Norway,Norway,https://financing.desa.un.org/sites/default/fi...,March 2024 Statement on behalf of Norway Writt...,"{'neg': 0.019, 'neu': 0.871, 'pos': 0.11, 'com...",en,0.9994,0.110,0.019,0.871,2187
86,86,"Michel, Bob",Michel 2C Bob,https://financing.desa.un.org/sites/default/fi...,Dispute resolution under a Framework Conventio...,"{'neg': 0.078, 'neu': 0.807, 'pos': 0.114, 'co...",en,0.9976,0.114,0.078,0.807,2185
95,95,Tax Justice Network (TJN),Tax Justice Network 28TJN 29,https://financing.desa.un.org/sites/default/fi...,Tax Justice Network submission to the Call for...,"{'neg': 0.033, 'neu': 0.805, 'pos': 0.162, 'co...",en,0.9998,0.162,0.033,0.805,2177
67,67,Center for Economic and Social Rights (CESR),Center for Economic and Social Rights 28CESR 29,https://financing.desa.un.org/sites/default/fi...,1 Ad Hoc Committee to Draft Terms of Reference...,"{'neg': 0.05, 'neu': 0.845, 'pos': 0.106, 'com...",en,0.9988,0.106,0.050,0.845,2171


In [ ]:
# !zip -r /content/all_files.zip /content/*
# from google.colab import files
# files.download('/content/all_files.zip')